In [1]:
# General Dependencies

import os
import math
import numpy as np
import torch

# Lines below are reloading modules every time you run a cell,
# such that this cell doesn't have to be re-run every time something
# changes in the source code.
#%load_ext autoreload 
#%autoreload 2

In [2]:
"""
Run from Colab or local
"""

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    ROOT_PATH = "/content/gdrive/MyDrive/502-796-Projects/502/deneme-1"
    %cd ./gdrive/MyDrive/502-796-Projects/502/deneme-1
except:
    ROOT_PATH = os.curdir


DATA_PATH = os.path.join(ROOT_PATH, "Datasets")

In [3]:
# Source dependencies
from data.dataset import FSSDataset
from common.vis import Visualizer
from common.evaluation import Evaluator

## Prepare PASCAL-$5^i$ Dataset

In [4]:
# STEP 1: Download PASCAL VOC2012 devkit (train/val data): (uncomment lines below)
# ------------------------------------------------------------------------------
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
#!tar -xvf 'VOCtrainval_11-May-2012.tar' -C ./Datasets/ 

# (or instead of wget, use directly the link to download)
# STEP 2: Place "VOC2012" folder from downloaded "VOCdevkit" folder under a "Datasets" folder.
# ------------------------------------------------------------------------------
#!mv Datasets/VOCdevkit/VOC2012 Datasets/VOC2012

# STEP 3: Download extended annotations from here
#!wget https://drive.google.com/file/d/10zxG2VExoEZUeyQl_uXga2OWHjGeZaf2/view

# STEP 4: Extract Put the downloaded extension under "Datasets/VOC2012/"


In [5]:
# Dataset initialization 
# TODO: does paper mention about image size? (the reference code is using im_size 400)
# TODO: add disclaimer to dataset files https://github.com/juhongm999/hsnet/blob/main/train.py
FSSDataset.initialize(img_size=128, datapath=DATA_PATH, use_original_imgsize=False)
dataloader_trn = FSSDataset.build_dataloader(benchmark='pascal', bsz=4, nworker=1, fold=0, split='trn')
dataloader_val = FSSDataset.build_dataloader(benchmark='pascal', bsz=4, nworker=1, fold=0, split='val')

Total (trn) images are : 11394
Total (val) images are : 346


## deneme

In [6]:
Visualizer.initialize(True)
Evaluator.initialize()

In [7]:
""" Code below visualizes prediction (to be used in testing)

for idx, batch in enumerate(dataloader_trn):
        # 1. Hypercorrelation Squeeze Networks forward pass
        #batch = utils.to_cuda(batch)
        #pred_mask = model.module.predict_mask_nshot(batch, nshot=nshot)

        #assert pred_mask.size() == batch['query_mask'].size()
        pred_mask = batch['query_mask']

        # 2. Evaluate prediction
        area_inter, area_union = Evaluator.classify_prediction(pred_mask.clone(), batch)
        
        #average_meter.update(area_inter, area_union, batch['class_id'], loss=None)
        #average_meter.write_process(idx, len(dataloader), epoch=-1, write_batch_idx=1)

        # Visualize predictions
        if Visualizer.visualize:
            Visualizer.visualize_prediction_batch(batch['support_imgs'], batch['support_masks'],
                                                  batch['query_img'], batch['query_mask'],
                                                  pred_mask, batch['class_id'], idx,
                                                  area_inter[1].float() / area_union[1].float())
"""


" Code below visualizes prediction (to be used in testing)\n\nfor idx, batch in enumerate(dataloader_trn):\n        # 1. Hypercorrelation Squeeze Networks forward pass\n        #batch = utils.to_cuda(batch)\n        #pred_mask = model.module.predict_mask_nshot(batch, nshot=nshot)\n\n        #assert pred_mask.size() == batch['query_mask'].size()\n        pred_mask = batch['query_mask']\n\n        # 2. Evaluate prediction\n        area_inter, area_union = Evaluator.classify_prediction(pred_mask.clone(), batch)\n        \n        #average_meter.update(area_inter, area_union, batch['class_id'], loss=None)\n        #average_meter.write_process(idx, len(dataloader), epoch=-1, write_batch_idx=1)\n\n        # Visualize predictions\n        if Visualizer.visualize:\n            Visualizer.visualize_prediction_batch(batch['support_imgs'], batch['support_masks'],\n                                                  batch['query_img'], batch['query_mask'],\n                                          

## Extract features

In [8]:
from model import feature

In [9]:
from torchvision.models import resnet
from functools import reduce
from operator import add

for idx, batch in enumerate(dataloader_trn):

    
    query_img = batch['query_img']
    
    backbone = resnet.resnet50(pretrained=True)
    
    
    nbottlenecks = [3, 4, 6, 3] # Resnet50
    
    feat_ids = list(range(4, 17))
    bottleneck_ids = reduce(add, list(map(lambda x: list(range(x)), nbottlenecks)))
    
    lids = reduce(add, [[i + 1] * x for i, x in enumerate(nbottlenecks)])
    stack_ids = torch.tensor(lids).bincount().__reversed__().cumsum(dim=0)[:3]
    backbone.eval()
    
    query_feats = feature.extract_feat_res(query_img, backbone,  feat_ids, bottleneck_ids, lids)
    
    break

C:\Users\bartu\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\bartu\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
print(len(query_feats))
 


13


In [11]:
for feat in query_feats:
    print(feat.shape)

torch.Size([4, 512, 16, 16])
torch.Size([4, 512, 16, 16])
torch.Size([4, 512, 16, 16])
torch.Size([4, 512, 16, 16])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 1024, 8, 8])
torch.Size([4, 2048, 4, 4])
torch.Size([4, 2048, 4, 4])
torch.Size([4, 2048, 4, 4])
